# Assignment 1

On this assignment, you’ll describe how a simple recommendation system can be crafted to leverage user ratings and generate individual recommendations for movies, products, and any thing else that can be rated.

## Submission
Submit your .ipynb file with a pdf of your assignment solution through the assignment submission portal on Canvas. No need to zip the files.

## **Code**

In [3]:
import pandas as pd

In [9]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('../ml-1m/users.dat', sep='::',
                      header=None, names=unames, engine='python', encoding='latin-1')

rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('../ml-1m/ratings.dat', sep='::',
                        header=None, names=rnames, engine='python', encoding='latin-1')

mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('../ml-1m/movies.dat', sep='::',
                       header=None, names=mnames, engine='python', encoding='latin-1')


In [11]:
users.info()
ratings.info()
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     6040 non-null   int64 
 1   gender      6040 non-null   object
 2   age         6040 non-null   int64 
 3   occupation  6040 non-null   int64 
 4   zip         6040 non-null   object
dtypes: int64(3), object(2)
memory usage: 236.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   user_id    1000209 non-null  int64
 1   movie_id   1000209 non-null  int64
 2   rating     1000209 non-null  int64
 3   timestamp  1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    ------------

In [41]:
# Create one merged DataFrame
movie_ratings = pd.merge(movies, ratings)
lens = pd.merge(movie_ratings, users)
lens.head()

,movie_id,title,genres,user_id,rating,timestamp,gender,age,occupation,zip
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268,F,1,10,48067
1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,1,5,978824351,F,1,10,48067
2,150,Apollo 13 (1995),Drama,1,5,978301777,F,1,10,48067
3,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1,4,978300760,F,1,10,48067
4,527,Schindler's List (1993),Drama|War,1,5,978824195,F,1,10,48067


### Question 1 

**[10 pts] An aggregate of movie ratings by men of age above 25 for each particular genre, e.g., Action, Adventure, Drama, Science Fiction, ... Note, Action|Drama|Thriller’ is not considered a unique genre. The movie that has a genre like this belongs to all three genres.**

In [42]:
test1 = lens.copy()

In [43]:
# make the genres column a list
test1.genres = test1.genres.str.split('|')

In [44]:
test1 = test1.explode('genres')
test1.head()

,movie_id,title,genres,user_id,rating,timestamp,gender,age,occupation,zip
0,1,Toy Story (1995),Animation,1,5,978824268,F,1,10,48067
0,1,Toy Story (1995),Children's,1,5,978824268,F,1,10,48067
0,1,Toy Story (1995),Comedy,1,5,978824268,F,1,10,48067
1,48,Pocahontas (1995),Animation,1,5,978824351,F,1,10,48067
1,48,Pocahontas (1995),Children's,1,5,978824351,F,1,10,48067


In [45]:
above_25 = test1[test1['age'] > 25]
above_25.head()

,movie_id,title,genres,user_id,rating,timestamp,gender,age,occupation,zip
53,1,Toy Story (1995),Animation,6,4,978237008,F,50,9,55117
53,1,Toy Story (1995),Children's,6,4,978237008,F,50,9,55117
53,1,Toy Story (1995),Comedy,6,4,978237008,F,50,9,55117
54,17,Sense and Sensibility (1995),Drama,6,4,978236383,F,50,9,55117
54,17,Sense and Sensibility (1995),Romance,6,4,978236383,F,50,9,55117


In [50]:
result1 = above_25.groupby('genres').agg({'rating':['max', 'min', 'mean', 'size']})

rating                      
              max min      mean    size
genres                                 
Action          5   1  3.555491   95015
Adventure       5   1  3.550061   50998
Animation       5   1  3.746264   14121
Children's      5   1  3.534032   25182
Comedy          5   1  3.591168  132228

In [60]:
result1

rating                      
               max min      mean    size
genres                                  
Action           5   1  3.555491   95015
Adventure        5   1  3.550061   50998
Animation        5   1  3.746264   14121
Children's       5   1  3.534032   25182
Comedy           5   1  3.591168  132228
Crime            5   1  3.761627   29437
Documentary      5   1  3.949221    3210
Drama            5   1  3.818925  150247
Fantasy          5   1  3.513150   12776
Film-Noir        5   1  4.103929    9112
Horror           5   1  3.251837   27756
Musical          5   1  3.757801   17626
Mystery          5   1  3.770696   17697
Romance          5   1  3.696242   60265
Sci-Fi           5   1  3.508494   60927
Thriller         5   1  3.647715   71550
War              5   1  3.948600   31245
Western          5   1  3.705617   10432

### Question 2

**[5 pts] The top 5 ranked movies by the most number of ratings (not the highest rating).**

In [56]:
test2 = lens.copy()

In [57]:
result2 = test2.groupby('title').size().sort_values(ascending=False)[:5]

In [59]:
result2

title
American Beauty (1999)                                   3428
Star Wars: Episode IV - A New Hope (1977)                2991
Star Wars: Episode V - The Empire Strikes Back (1980)    2990
Star Wars: Episode VI - Return of the Jedi (1983)        2883
Jurassic Park (1993)                                     2672
dtype: int64

### Question 3

**[20 pts] Average movie ratings between users of different age groups (<18, 18-30, 30-50, 50-70, 70>)**

### Question 4

**[25 pts] Pick a movie of your choice and for all movies of the same year, provide a breakdown of the number of unique movies rated by 3 ranges of age of reviewers (a) under 18 (b) 19 to 45 (c) Above 45.**

### Question 5

**[20 pts] A function that takes in a user_id and a movie_id, and returns a list of all the other movies that the user rated similarly to the given movie, i.e. with the same rating. Demonstrate that your function works.**

### Question 6

**[20 pts] Some other statistic, figure, aggregate, or plot that you created using this dataset, along with a short description of what interesting observations you derived from it.**